In [1]:
import pandas as pd
import numpy as np
import ast
import datetime
import os
import seaborn as sns
from scipy import stats
from sklearn import preprocessing

In [2]:
from statsmodels.tsa.arima_model import ARIMA

In [3]:
import ast
d1 = pd.DataFrame(pd.read_csv('tps2000000000069',sep='|',header=None).iloc[:,0]
         .apply(ast.literal_eval).tolist())

In [4]:
d2=pd.DataFrame(pd.read_csv('df_8.csv'))

In [5]:
d=pd.concat([d1,d2],ignore_index=True)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [6]:
#drop duplicate
d=d.drop_duplicates()

In [7]:
#convert to_datetime
d['localtime'] = d['localtime'].map(lambda x: x.rstrip('UTC'))
d['localtime']=d['localtime'].apply(pd.to_datetime)

In [10]:
d['date'] = [d.date() for d in d['localtime']]
d['time'] = [d.time() for d in d['localtime']]

In [11]:
d['date']=d['date'].apply(pd.to_datetime)

In [12]:
# label grid

In [13]:
bottomLeft = (-22.81583,-43.24794)

topRight = (-22.81315,-43.24410)

n=20

test_grid=d.copy()

lat_steps = np.linspace(bottomLeft[0], topRight[0], n+1)
lon_steps = np.linspace(bottomLeft[1], topRight[1], n+1)

test_grid['col'] = np.searchsorted(lat_steps, test_grid['lat'])
test_grid['row'] = np.searchsorted(lon_steps, test_grid['lng'])

num=0
rows=[]
cols=[]
nums=[]
for col in range(20):
    for row in range(20):
        num=num+1
        rows.append(row+1)
        cols.append(col+1)
        nums.append(num)
label=pd.DataFrame([rows,cols,nums]).transpose()
label.columns=['row','col','num']
grid_label=pd.merge(test_grid,label,on=['row','col'])
grid_label


#grid range: co
lat_stride = lat_steps[1] - lat_steps[0]
lon_stride = lon_steps[1] - lon_steps[0]
co=[]
for lat in lat_steps[:-1]:
    for lon in lon_steps[:-1]:
            # Define dimensions of box in grid
        upper_left = [lon, lat + lat_stride]
        upper_right = [lon + lon_stride, lat + lat_stride]
        lower_right = [lon + lon_stride, lat]
        lower_left = [lon, lat]
        coordinates = [upper_left,upper_right,lower_right,lower_left]
        co.append(coordinates)

grid_label.index=grid_label["localtime"]
grid_label

,Building,ClientMacAddr,Level,Unnamed: 0,lat,lng,localtime,date,time,col,row,num
localtime,,,,,,,,,,,,
2019-12-30 10:14:38.543,TPS2,7c:8b:b5:f0:34:7b,Level 1,NaN,-22.814944,-43.245837,2019-12-30 10:14:38.543,2019-12-30,10:14:38.543000,7,11,131
2019-12-30 10:14:42.831,TPS2,7c:a1:77:8b:32:24,Level 1,NaN,-22.815013,-43.245856,2019-12-30 10:14:42.831,2019-12-30,10:14:42.831000,7,11,131
2019-12-30 10:14:54.301,TPS2,10:d0:7a:c9:35:ed,Level 1,NaN,-22.815015,-43.245901,2019-12-30 10:14:54.301,2019-12-30,10:14:54.301000,7,11,131
2019-12-30 10:14:58.250,TPS2,44:e4:ee:02:61:d5,Level 1,NaN,-22.814990,-43.245839,2019-12-30 10:14:58.250,2019-12-30,10:14:58.250000,7,11,131
2019-12-30 10:15:03.544,TPS2,da:a1:19:ae:83:a8,Level 1,NaN,-22.815025,-43.246011,2019-12-30 10:15:03.544,2019-12-30,10:15:03.544000,7,11,131
2019-12-30 10:15:04.509,TPS2,7c:a1:77:8b:32:24,Level 1,NaN,-22.814993,-43.245926,2019-12-30 10:15:04.509,2019-12-30,10:15:04.509000,7,11,131
2019-12-30 10:15:22.262,TPS2,00:cd:fe:80:51:83,Level 1,NaN,-22.815017,-43.245972,2019-12-30 10:15:22.262,2019-12-30,10:15:22.262000,7,11,131
2019-12-30 10:15:43.598,TPS2,a8:a6:68:bc:ec:16,Level 1,NaN,-22.815023,-43.245872,2019-12-30 10:15:43.598,2019-12-30,10:15:43.598000,7,11,131
2019-12-30 10:15:47.791,TPS2,90:73:5a:5c:fb:fb,Level 1,NaN,-22.814975,-43.245878,2019-12-30 10:15:47.791,2019-12-30,10:15:47.791000,7,11,131


In [20]:
d_test=grid_label.copy()

In [22]:
#August

In [23]:
d8=(d_test['date']>='2019-08-01')&(d_test['date']<='2019-08-31')

In [24]:
d_8=d_test.loc[d8]

In [25]:
d_8.head()

,Building,ClientMacAddr,Level,Unnamed: 0,lat,lng,localtime,date,time,col,row,num
localtime,,,,,,,,,,,,
2019-08-01 04:39:14.136,TPS2,3a:ea:9f:4d:00:5b,Level 2,NaN,-22.814995,-43.245970,2019-08-01 04:39:14.136,2019-08-01,04:39:14.136000,7,11,131
2019-08-01 04:39:52.613,TPS2,dc:bf:e9:61:79:68,Level 2,NaN,-22.814978,-43.245940,2019-08-01 04:39:52.613,2019-08-01,04:39:52.613000,7,11,131
2019-08-01 04:40:19.738,TPS2,a0:56:f3:6c:d3:83,Level 2,NaN,-22.815007,-43.245961,2019-08-01 04:40:19.738,2019-08-01,04:40:19.738000,7,11,131
2019-08-01 05:11:24.704,TPS2,60:30:d4:25:cb:6a,Level 2,NaN,-22.814975,-43.245835,2019-08-01 05:11:24.704,2019-08-01,05:11:24.704000,7,11,131
2019-08-01 05:20:33.233,TPS2,e2:a0:7b:d5:91:ec,Level 2,NaN,-22.814985,-43.245922,2019-08-01 05:20:33.233,2019-08-01,05:20:33.233000,7,11,131


In [26]:
#September:actual frequenct

In [40]:
d9=d_test[d_test["date"]=="2019-09-01"]
d9.head()

,Building,ClientMacAddr,Level,Unnamed: 0,lat,lng,localtime,date,time,col,row,num
localtime,,,,,,,,,,,,
2019-09-01 07:05:42.303,TPS2,28:83:35:a6:f8:e9,Level 2,675079.0,-22.814955,-43.245857,2019-09-01 07:05:42.303,2019-09-01,07:05:42.303000,7,11,131
2019-09-01 07:05:42.303,TPS2,28:83:35:a6:f8:e9,Level 2,675080.0,-22.814955,-43.245857,2019-09-01 07:05:42.303,2019-09-01,07:05:42.303000,7,11,131
2019-09-01 07:15:56.474,TPS2,8a:6f:60:df:6b:0d,Level 2,682725.0,-22.814974,-43.246002,2019-09-01 07:15:56.474,2019-09-01,07:15:56.474000,7,11,131
2019-09-01 07:15:56.474,TPS2,8a:6f:60:df:6b:0d,Level 2,682726.0,-22.814974,-43.246002,2019-09-01 07:15:56.474,2019-09-01,07:15:56.474000,7,11,131
2019-09-01 07:31:29.692,TPS2,28:83:35:a6:f8:e9,Level 2,694727.0,-22.814987,-43.246001,2019-09-01 07:31:29.692,2019-09-01,07:31:29.692000,7,11,131


In [49]:
def test(d9,t1,t2):
    grid=list() 
    for i in range(401):
        dt=d9[d9.num==i]
        hour=dt.resample(rule = '1H').count()
        hour=pd.DataFrame(hour.Building) 
        b=hour.between_time(t1,t2).values
        grid.append(b)
    true=pd.DataFrame(grid,columns=["actual frequency"])
    true=true.iloc[1:]
    return true

In [50]:
dactual = pd.DataFrame()
for i in range(0,24):
    t1 = "%d:00:00" %i
    t2 = "%d:59:59" %i
    ac = test(d9,t1,t2)
    ac['hour'] = i
    dactual = dactual.append(a)

In [51]:
dactual['grid'] = dactual.index
dactual.to_csv(r'level2_actual.csv')

In [28]:
# specific time period with 400 grids

In [34]:
def predict(data,t1,t2):
    grid_pre=list()
    grid_mean=list()    
    for i in range(401):
        dh=data[data.num==i]
        if len(dh)==0:
            y_pre=np.array([0])
        else:
            dh=dh.resample(rule = '1H').count()
            dh=pd.DataFrame(dh.Building)  
            t=dh.between_time(t1,t2)
            a=t.values.mean()
            if len(t) <3 or a <1:
                if len(t)==0:
                    y_pre=np.array([0])
                else:
                    y_pre=np.array([int(np.mean(t.values))])
            else:
                model = ARIMA(t, order=(1,0,0))
                model_fit = model.fit()
                yfore = model_fit.forecast()
                if yfore[0]>0:
                    y_pre=yfore[0]
                else:
                    y_pre=np.array([0])
        grid_pre.append(y_pre)
    grid_pre=pd.DataFrame(grid_pre,columns=["predicted frequency"])
    grid_pre["predicted frequency"]=grid_pre["predicted frequency"].astype(int)
    grid_pre=grid_pre.iloc[1:]
    return grid_pre

In [35]:
q=predict(d_8,"10:00:00","10:59:59")
q[q["predicted frequency"]>0]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: Mean of empty slice.
  if sys.path[0] == '':
/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/anaconda3/lib/python3.7/site-packages/scipy/signal/signaltools.py:1341: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out_full[ind] += zi
/anaconda3/lib/python3.7/site-packages/scipy/signal/signaltools.py:1344: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.

,predicted frequency
7,1359
8,2126
9,1211
10,120
11,124
12,65
13,440
27,379
28,20
29,1493


In [36]:
df = pd.DataFrame()
for i in range(0,24):
    s1 = "%d:00:00" %i
    s2 = "%d:59:59" %i
    a = predict(d_8,s1,s2)
    a['hour'] = i
    df = df.append(a)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: Mean of empty slice.
  if sys.path[0] == '':
/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/anaconda3/lib/python3.7/site-packages/scipy/signal/signaltools.py:1341: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out_full[ind] += zi
/anaconda3/lib/python3.7/site-packages/scipy/signal/signaltools.py:1344: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.

In [37]:
df['grid'] = df.index

In [38]:
df.to_csv(r'level2_pred.csv')

In [39]:
len(d)

23804979

In [ ]:
# one grid with 24 hours

In [ ]:
hourlist=[['00:00:00',"00:59:59"],["01:00:00","01:59:59"],["02:00:00","02:59:59"],
          ["03:00:00","03:59:59"],["04:00:00","05:59:59"],["05:00:00","05:59:59"],
          ["06:00:00","06:59:59"],["07:00:00","07:59:59"],["08:00:00","08:59:59"],
          ["09:00:00","09:59:59"],["10:00:00","10:59:59"],["11:00:00","11:59:59"],
          ["12:00:00","12:59:59"],["13:00:00","13:59:59"],["14:00:00","14:59:59"],
          ["15:00:00","15:59:59"],["16:00:00","16:59:59"],["17:00:00","17:59:59"],
          ["18:00:00","18:59:59"],["19:00:00","19:59:59"],["20:00:00","20:59:59"],
          ["21:00:00","21:59:59"],["22:00:00","22:59:59"],["23:00:00","23:59:59"]]

In [ ]:
def predicttime(data,grid):
    d=data[data.label==grid]
    d.index=d["localtime"]
    dh=d.resample(rule = '1H').count()
    dh=pd.DataFrame(dh.Building)
    dh.columns=["frequency"]
    grid_pre=list()
    grid_mean=list()
    for i,j in hourlist:
        t=dh.between_time(i,j)
        model = ARIMA(t, order=(0,1,1))
        model_fit = model.fit()
        yfore = model_fit.forecast()
        if yfore[0]>0:
            y_pre=yfore[0]
        else:
            y_pre=np.array([0])
        y_mean=t.mean().frequency
        grid_pre.append(y_pre)
        grid_mean.append(y_mean)
    grid_pre=pd.DataFrame(grid_pre,columns=["predicted frequency"])
    grid_mean=pd.DataFrame(grid_mean,columns=["average frequency"]) 
    grid_pre["average frequency"]=grid_mean
    return grid_pre

In [ ]:
predicttime(d_8,315)